In [20]:
#Import Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import os
import time

In [21]:
dashboardUrl = "https://lms.lpuonline.com/"

In [22]:
#Load the web driver and get the url
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(dashboardUrl)

In [23]:
load_dotenv()
username = os.environ.get("registrationId")
password = os.environ.get("password")
print(type(username), type(password))

<class 'str'> <class 'str'>


In [24]:
usernameInput = driver.find_element(By.ID, "username")
passwordInput = driver.find_element(By.ID, "Password")
loginButton = driver.find_element(By.ID, "LoginButton")

In [25]:
usernameInput.send_keys(username)
passwordInput.send_keys(password)
loginButton.click()

In [26]:
totalSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card").__len__()

In [27]:
def lectureHandler(inPlay = False):
    totalLectures = (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli").__len__()
    if totalLectures == 0:
        raise Exception("Lectures not loaded")
    for currentLectureIndex in range(totalLectures):
        print("currentLectureIndex ", currentLectureIndex, "total lectures in current unit is ", totalLectures)
        try:
            nextLecture =  (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli")[currentLectureIndex]
        except: 
            nextLecture = driver.find_element(By.CLASS_NAME, "inPlay").find_elements(By.CLASS_NAME, "videoli")[currentLectureIndex]
        nextLecture.find_element(By.CLASS_NAME, "cc-a").click()
        time.sleep(5)
        driver.execute_script(open("./videoHandler.js").read())
        time.sleep(10)

In [28]:
for currentSubjectIndex in range(totalSubjects):
    print("\n\ncurrentSubjectIndex ", currentSubjectIndex)
    allSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card")
    allSubjects[currentSubjectIndex].find_element(By.TAG_NAME, 'a').click()
    totalRemainingUnits = driver.find_element(By.CLASS_NAME, "unit").find_elements(By.CLASS_NAME, "collapse").__len__()
    print("\ntotalRemainingUnit", totalRemainingUnits)
    try:
        lectureHandler(True)
    except:
        time.sleep(20)
        lectureHandler(True)
    currentRemainingUnitIndex = 1
    while currentRemainingUnitIndex < totalRemainingUnits:
        currentRemainingUnitIndex = currentRemainingUnitIndex + 1
        print("\ncurrentUnitIndex ", currentRemainingUnitIndex)
        currentUnitTitle = driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "lecture-title")
        WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable(currentUnitTitle))
        currentUnitTitle.click()
        WebDriverWait(driver, 20).until(lambda driver: expected_conditions.presence_of_element_located((By.CLASS_NAME, "videoli"))(currentUnitTitle.find_element(By.XPATH, "./following-sibling::div")))
        try:
           lectureHandler()
        except Exception as e:
            if e == "Lectures not loaded":
                driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "lecture-title").click()
                time.sleep(5)
                lectureHandler()
    driver.back()
print("All Good :D")



currentSubjectIndex  0

currentUnitIndex  0 totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  3
currentLectureIndex  1 total lectures in current unit is  3
currentLectureIndex  2 total lectures in current unit is  3


currentSubjectIndex  1

currentUnitIndex  0 totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  1


currentSubjectIndex  2

currentUnitIndex  0 totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  2
currentLectureIndex  1 total lectures in current unit is  2


currentSubjectIndex  3

currentUnitIndex  0 totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  10
currentLectureIndex  1 total lectures in current unit is  10
currentLectureIndex  2 total lectures in current unit is  10
currentLectureIndex  3 total lectures in current unit is  10
currentLectureIndex  4 total lectures in current unit is  10
currentLectureIndex  5 total lectures in current unit is  10
curren

ElementClickInterceptedException: Message: element click intercepted: Element <div id="lecture-title_2" class="lecture-title collapsed" data-toggle="collapse" href="#collapse2" aria-expanded="false">...</div> is not clickable at point (1193, 28). Other element would receive the click: <div class="col-md-12 text-center topresmobi">...</div>
  (Session info: chrome=121.0.6167.86)
Stacktrace:
	GetHandleVerifier [0x00007FF700205E42+3538674]
	(No symbol) [0x00007FF6FFE24C02]
	(No symbol) [0x00007FF6FFCD5AEB]
	(No symbol) [0x00007FF6FFD233E4]
	(No symbol) [0x00007FF6FFD21122]
	(No symbol) [0x00007FF6FFD1E884]
	(No symbol) [0x00007FF6FFD1D5A9]
	(No symbol) [0x00007FF6FFD11950]
	(No symbol) [0x00007FF6FFD3F0AA]
	(No symbol) [0x00007FF6FFD1124A]
	(No symbol) [0x00007FF6FFD3F2C0]
	(No symbol) [0x00007FF6FFD5BDE3]
	(No symbol) [0x00007FF6FFD3EE53]
	(No symbol) [0x00007FF6FFD0F514]
	(No symbol) [0x00007FF6FFD10631]
	GetHandleVerifier [0x00007FF700236CAD+3738973]
	GetHandleVerifier [0x00007FF70028C